In [ ]:
# とりあえず動くものを実装する
from pathlib import Path
import asyncio
import pickle
import datetime
import json

from pydantic import BaseModel
import polars as pl

import stock
import data_fetcher
import auto_trader

In [ ]:
symbol = "BTC_JPY"
pips = 0.00001
interval = datetime.timedelta(minutes=15)
data_length = 30

In [ ]:
#fetcher = auto_trader.data_fetcher.GMODataFetcher(symbol)
# modelの読み込み
model_file = Path("/home/kitamura/work/stock/notebooks/model.pkl")
with open(model_file, "rb") as f:
    model = pickle.load(f)

In [ ]:
# 初期データの用意
df = auto_trader.strategy.fetch_initial_df(symbol, interval, min_length=data_length).sort(pl.col("datetime"))[-data_length:]

In [ ]:
def post_order(symbol: str, price: float, volume):
    side = "BUY" if volume > 0  else "SELL"
    if price > 0:
        params = {
            "symbol": symbol,
            "side": side,
            "executionType": "LIMIT",
            "timeInForce": "FAS",
            "price": int(price),
            "size": abs(volume)
        }
    else:
        params = {
            "symbol": symbol,
            "side": side,
            "executionType": "MARKET",
            "timeInForce": "FAS",
            "size": abs(volume)
        }
    res = auto_trader.utils.gmo.private_api(
        "/v1/order", parameters=params, method="POST"
    )
    return res

def get_available_amount():
    res = auto_trader.utils.gmo.private_api(
        "/v1/account/margin", parameters={}, method="GET"
    )
    if res["status"] == 0:
        return int(res["data"]["availableAmount"])
    raise RuntimeError("Error returned: {}".format(json.dumps(res, indent=2)))


In [ ]:
async def wait_until(dt):
    # sleep until the specified datetime
    now = datetime.datetime.now()
    await asyncio.sleep((dt - now).total_seconds())


def is_order_finished(order_id):
    res = auto_trader.utils.gmo.private_api("/v1/orders", parameters={"orderId": order_id}, method="GET")
    return res["data"]["status"] in ["CANCELED", "EXECUTED", "EXPIRED"]
    
def calc_executed_volume(order_id):
    """`order_id`の注文で約定済みの数量を求める
    """
    executed_volume = 0.0
    res = auto_trader.utils.gmo.private_api("/v1/orders", parameters={"orderId": order_id}, method="GET")
    for data in res["data"]["list"]:
        if data["side"] == "BUY":
            executed_volume += float(data["size"])
        elif data["side"] == "SELL":
            executed_volume -= float(data["size"])
    return executed_volume

class Order(BaseModel):
    symbol: str
    order_id: int
    losscut_price: float
    close_order_id: int = -1
    closed: bool = False

    @staticmethod
    def new_order(symbol: str, price: float, volume: float, losscut_price: float):
        order = post_order(symbol, price, volume)
        return Order(symbol=symbol, order_id=order, losscut_price=losscut_price)
    
    def is_closed(self):
        """注文がすべて終了状態で、持ち高が0の状態の場合はTrue、そうでない場合はFalse"""
        if self.closed:
            return self.closed
        
        def _is_closed():
            if not is_order_finished(self.order_id): 
                return False  # 注文がまだ有効な場合
            executed = calc_executed_volume(self.order_id) 
            if abs(executed) < 1e-5:  
                return True # 持ち高が0の場合
            if self.close_order_id < 0:  
                return False # 持ち高がある状態で、反対取引が発行されていない
            if not is_order_finished(self.close_order_id):  
                return False # 反対取引が有効な場合
            executed -= calc_executed_volume(self.close_order_id)
            if abs(executed) < 1e-5:
                return True  # 反対取引が約定済みで持ち高が0の場合
            return False  # 反対取引は約定済みだが持ち高がまだある場合
        
        self.closed = _is_closed()
        return self.closed

    def losscut(self):
        """losscutを実行する
        Return:
            bool : Trueの場合は持ち高精算済み、そうでない場合（losscut注文発行）はFalse
        """
        self.cancel_order() # 注文が有効な場合はキャンセル
        executed = calc_executed_volume(self.order_id) 

        if self.close_order_id > 0:  # 反対取引の注文が発行されている場合
            self.cancel_order(self.close_order_id)
            executed -= calc_executed_volume(self.order_id)

        if abs(executed) < 1e-5:
            self.closed = True
            return True
        self.close_order_id = post_order(symbol=self.symbol, price=-1.0, volume=-executed)  # losscutは成り行きで実行
        return False
    
    def check_losscut(self, current_price: float):
        if self.is_closed():
            return 
        
        if current_price < self.losscut_price:
            self.losscut()

    def cancel_order(self, order_id = None):
        if order_id is None:
            order_id = self.order_id
        if not is_order_finished(order_id):
            auto_trader.utils.gmo.private_api("/v1/cancelOrder", parameter={"orderId", order_id}, method="POST")

    def get_current_position(self):
        executed = calc_executed_volume(self.order_id) 
        if self.close_order_id > 0:  # 反対取引の注文が発行されている場合
            executed -= calc_executed_volume(self.order_id)
        return executed

    def update_target_price(self, target_price: float):
        """利益確定注文の価格を変更する"""
        if self.is_closed():
            return 
        
        executed = calc_executed_volume(self.order_id)
        if self.close_order_id > 0:
            self.cancel_order(self.close_order_id)
            executed -= calc_executed_volume(self.close_order_id)

        self.close_order_id = post_order(self.symbol, target_price, executed)

In [ ]:
current = datetime.datetime.now()
next_wall_minute = (current.minute // 15  + 1) * 15
offset = 1 if next_wall_minute > 59 else 0
next_wall_minute = next_wall_minute % 60
next_wall = datetime.datetime(
    year=current.year, month=current.month, day=current.day, hour=current.hour + offset, minute=next_wall_minute
)

orders: list[Order] = []
fut = asyncio.sleep(10)
while True:
    await fut
    fut = asyncio.sleep(10)

    # 最新の価格を更新
    latest_data = auto_trader.utils.gmo.public_api("/v1/ticker")["data"]
    for order in orders:
        for data in latest_data:
            if data["symbol"] == order.symbol:
                order.check_losscut(data["last"])
                break

    # next wallに到達した場合の処理
    if next_wall < datetime.datetime.now():
        next_wall += interval  # next wallの更新
    
        # 特徴量の計算、モデルの実行
        df = auto_trader.strategy.fetch_initial_df(symbol, interval, min_length=data_length).sort(pl.col("datetime"))[-data_length - 1:-1]
        df = stock.crypto.feature.calc_features(df)
        feat = df.select(*auto_trader.strategy.Strategy0.train_features)
        preds = model.predict(feat)

        # 発注済みの注文の目標株価を更新
        target_buy_price = df["close"][-1] - df["ATR"][-1] * 0.5
        target_sell_price = df["close"][-1] + df["ATR"][-1] * 0.5
        for order in orders:
            if order.get_current_position() < 0:
                order.update_target_price(target_buy_price)
            else:
                order.update_target_price(target_sell_price)

                
        if preds[-1] > 0: # モデルのスコアが良い場合は新規注文
        
        # 注文
        order = post_order(symbol, target_price, volume * pips)
        if order["status"] == 0:
            buy_orders.append(order["data"])

In [ ]:
# 状態取得
res = auto_trader.utils.gmo.private_api(
    "/v1/activeOrders", parameters={"symbol": "BTC"}, method="GET"
)
res

In [ ]:
# 取引余力
res = auto_trader.utils.gmo.private_api(
    "/v1/account/margin", parameters={}, method="GET"
)
res

In [ ]:
# 現物注文
params = {
    "symbol": "BTC",
    "side": "BUY",
    "executionType": "LIMIT",
    "timeInForce": "FAS",
    "price": int(df["low"].min() * 0.8),
    "size": 0.0001
}

res = auto_trader.utils.gmo.private_api(
    "/v1/order", parameters=params, method="POST"
)
res

In [ ]:
# 注文情報
order_id = "5673559057"
res = auto_trader.utils.gmo.private_api(
    "/v1/orders", parameters={"orderId": order_id}, method="GET"
)
res

In [ ]:
# 約定情報
res = auto_trader.utils.gmo.private_api(
    "/v1/executions", parameters={"orderId": order_id}, method="GET"
)
res